In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [9]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [10]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [11]:
from collections import defaultdict

In [12]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)


In [13]:
len(hashes), len(documents)

(947, 948)

In [14]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [15]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [16]:
import json

In [17]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [18]:
with open("documents-with-ids.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(json.dumps(data[:5], indent=4))

[
    {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?",
        "course": "data-engineering-zoomcamp",
        "id": "c02e79ef"
    },
    {
        "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",
        "section": "General course-related questions",
        "question": "Course - What are the prerequisites for this course?",
        "course": "data-engineering-zoomcamp",
        "id": "1f6520ca"
    },
    {


In [ ]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)

In [ ]:
def generate_questions(doc):  
    prompt = prompt_template.format(**doc)  
    # Tokenize input  
    inputs = tokenizer(prompt, return_tensors="pt")  
    # Generate response with use_cache set to False
    outputs = model.generate(**inputs, max_new_tokens=200, use_cache=False)  
    # Decode output  
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)    
    json_response = {"choices": [{"message": {"content": generated_text}}]}  
    return json_response["choices"][0]["message"]["content"]

In [ ]:
from tqdm.auto import tqdm

In [ ]:
results = {}

In [ ]:
# it will take lot for processing use results.bin for same result in next step
#for doc in tqdm(documents): 
#    doc_id = doc['id']
#    if doc_id in results:
#        continue
#
#    questions = generate_questions(doc)
#    results[doc_id] = questions

In [1]:
import pickle
import pandas as pd
import ast

In [2]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [3]:
# Check the type and structure of the loaded data
print(type(results))

<class 'dict'>


In [19]:
parsed_results = {key: ast.literal_eval(value) for key, value in results.items()}

In [20]:
doc_index = {d['id']: d for d in documents}

In [21]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [22]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [23]:
df.head(10)

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
5,Where can I find the prerequisites for this co...,data-engineering-zoomcamp,1f6520ca
6,How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
7,Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
8,What are the requirements for joining this cou...,data-engineering-zoomcamp,1f6520ca
9,Where is the list of prerequisites for the cou...,data-engineering-zoomcamp,1f6520ca


In [ ]:
df.to_csv('ground-truth-data.csv', index=False)